# netflow.py
Solve a multicommodity flow model using Gurobi Optimizer.


## List data
  0. Commodities (products)
  0. Nodes in the network
  0. Arcs in the network

In [1]:
from gurobipy import *

commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']
arcs = [
  ('Detroit', 'Boston'), ('Detroit', 'New York'), ('Detroit', 'Seattle'),
  ('Denver',  'Boston'), ('Denver',  'New York'), ('Denver',  'Seattle')]

## Indexed data
  0. Capacity: indexed by arcs
  0. Cost: indexed by commodities and arcs
  0. Inflow: indexed by commodities and nodes

In [2]:
capacity = {
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 }

cost = {
  ('Pencils', 'Detroit', 'Boston'):   10,  ('Pens', 'Detroit', 'Boston'):   20,
  ('Pencils', 'Detroit', 'New York'): 20,  ('Pens', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,  ('Pens', 'Detroit', 'Seattle'):  80,
  ('Pencils', 'Denver',  'Boston'):   40,  ('Pens', 'Denver',  'Boston'):   60,
  ('Pencils', 'Denver',  'New York'): 40,  ('Pens', 'Denver',  'New York'): 70,
  ('Pencils', 'Denver',  'Seattle'):  30,  ('Pens', 'Denver',  'Seattle'):  30 }

inflow = {
  ('Pencils', 'Detroit'):   50,  ('Pens', 'Detroit'):   60,
  ('Pencils', 'Denver'):    60,  ('Pens', 'Denver'):    40,
  ('Pencils', 'Boston'):   -50,  ('Pens', 'Boston'):   -40,
  ('Pencils', 'New York'): -50,  ('Pens', 'New York'): -30,
  ('Pencils', 'Seattle'):  -10,  ('Pens', 'Seattle'):  -30 }

In [3]:
for h in commodities:
    for c in nodes:
        print("Inflow " + h + " " + c + ": " + str(inflow[h,c]))

Inflow Pencils Detroit: 50
Inflow Pencils Denver: 60
Inflow Pencils Boston: -50
Inflow Pencils New York: -50
Inflow Pencils Seattle: -10
Inflow Pens Detroit: 60
Inflow Pens Denver: 40
Inflow Pens Boston: -40
Inflow Pens New York: -30
Inflow Pens Seattle: -30


## Create model, decision variables and objective

- Use `Model.addVars()` to add the decision variables
- With two arguments, it takes the cross product of the commodities and the arcs

In [4]:
m = Model('netflow')

flow = m.addVars(commodities, arcs, obj=cost, name="flow")

## Create constraints

- Use `Model.addConstrs()` to add the constraints
- Uses two **Python Generator expressions**
    - To generate an arc capacity constraint for every arc _i,j_
    - To generate a flow conservation constraint for every commodity _h_ and every node _j_
- Inside each constraint, uses the aggregate operator `tupledict.sum()` to compute the sum over only the matching elements

In [5]:
# Arc capacities
cap = m.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs), "cap")

# Flow conservation
node = m.addConstrs(
    (flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')
     for h in commodities for j in nodes), "node")

## Solve and print the flows

In [6]:
m.optimize()
m.printAttr('X')

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.500000000e+03

    Variable            X 
-------------------------
flow[Pencils,Detroit,Boston]           50 
flow[Pencils,Denver,New York]           50 
flow[Pencils,Denver,Seattle]           10 
flow[Pens,Detroit,Boston]           30 
flow[Pens,Detroit,New York]           30 
flow[Pens,Denver,Boston]           10 
flow[Pens,Denver,Seattle]           30 


## Output
Display the solution as a chart and as a table

In [7]:
from bokeh.charts import *
output_notebook()

keys = sorted(flow.keys())
data = {
    'arcs': ["%s-%s"% (i,j) for h,i,j in keys],
    'commodities': [h for h,i,j in keys],
    'flow': [flow[h,i,j].X for h,i,j in keys],
    'use': [flow[h,i,j].X/capacity[i,j] for h,i,j in keys],
}
bar = Bar(data, values='flow', label='arcs', stack='commodities', title="Network flow")
show(bar)

Loading BokehJS ...

In [8]:
import pandas as pd
mi = pd.MultiIndex.from_tuples(sorted(list(arcs)), names=('origin','destination'))
df = pd.DataFrame(index=mi, columns=commodities)
for h in commodities:
    for i,j in arcs:
        df[h][i,j] = flow[h,i,j].X

df

Pencils Pens
origin  destination             
Denver  Boston            0   10
        New York         50    0
        Seattle          10   30
Detroit Boston           50   30
        New York          0   30
        Seattle           0    0

## Debugging
Write the model as an LP file

In [9]:
m.write("netflow.lp")